In [1]:
import os
import pandas as pd
import gc
import numpy as np
from datasets import Dataset, DatasetDict
from sklearn.metrics import f1_score
import transformers
import time
import torch
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Trainer, TrainingArguments, GenerationConfig)
from peft import LoraConfig, get_peft_model, TaskType
from sqlglot import parse_one
from google.colab import drive
from peft import PeftModel

# --- Configuration & Data Loading ---

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Ensure drive is mounted for paths to work
try:
    drive.mount('/content/drive')
except:
    print("Drive already mounted or mount failed.")

MODEL_NAME = "gaussalgo/T5-LM-Large-text2sql-spider"
TRAIN_DATA_PATH = "/content/drive/MyDrive/QueryHub_FineTuning/train.csv"
TEST_DATA_PATH = "/content/drive/MyDrive/QueryHub_FineTuning/test.csv"
VAL_DATA_PATH = "/content/drive/MyDrive/QueryHub_FineTuning/val.csv"
OUTPUT_DIR = "/content/drive/MyDrive/QueryHub_FineTuning/text2sql_lora"
SAVE_DIR = "/content/drive/MyDrive/QueryHub_FineTuning/text2sql_lora" # Used in inference block

max_source_length = 128
max_target_length = 128

df_tr = pd.read_csv(TRAIN_DATA_PATH)
df_ts = pd.read_csv(TEST_DATA_PATH)
df_val = pd.read_csv(VAL_DATA_PATH)

# Strategy: Use a small sample for training evaluation (Probe)
df_tr_sample = df_tr.sample(n=5000, random_state=42)

# --- CRUCIAL FIX: Reduce validation set size used *during* training ---
df_val_sample_probe = df_val.sample(n=2000, random_state=42) # Small set for quick validation checks
# ---------------------------------------------------------------------

# Keep the large samples for final, comprehensive evaluation after training
df_val_sample_full = df_val.sample(n=20000, random_state=42)
df_ts_sample_full = df_ts.sample(n=5649, random_state=42)


del df_tr, df_ts, df_val
gc.collect()

dataset_tr = Dataset.from_pandas(df_tr_sample[["input_text", "sql"]])
# Use the small probe set for the Trainer's evaluation
dataset_val_probe = Dataset.from_pandas(df_val_sample_probe[["input_text", "sql"]])
# The full evaluation set is only used in the final inference section below

# --- Model Loading & Tokenizer ---

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if device == "cuda" else torch.float32,
    low_cpu_mem_usage=True
)
model.to(device)

model.gradient_checkpointing_enable()
tokenizer.truncation_side = "right"
model.config.use_cache = False

# --- Preprocessing ---

def preprocess_batch(batch):
    inputs = batch["input_text"]
    targets = batch["sql"]
    model_inputs = tokenizer(inputs, max_length=max_source_length, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding="max_length")
    labels_ids = labels["input_ids"]
    labels_ids = [[(token_id if token_id != tokenizer.pad_token_id else -100) for token_id in seq] for seq in labels_ids]
    model_inputs["labels"] = labels_ids
    return model_inputs

tokenized_train = dataset_tr.map(preprocess_batch, batched=True, remove_columns=dataset_tr.column_names, load_from_cache_file=True)
# Use the tokenized probe set for evaluation during training
tokenized_val_probe   = dataset_val_probe.map(preprocess_batch, batched=True, remove_columns=dataset_val_probe.column_names, load_from_cache_file=True)

# --- LoRA Setup & Trainer Arguments ---

lora_config = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.1, bias="none", task_type=TaskType.SEQ_2_SEQ_LM, target_modules=["q", "v"])
model = get_peft_model(model, lora_config)
model.to(device)
model.print_trainable_parameters()

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    learning_rate=3e-4,
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="token_f1",
    greater_is_better=True,
    report_to="none",
    eval_accumulation_steps=100,
    bf16=True,
    use_mps_device=False,
    gradient_checkpointing=True,
    dataloader_pin_memory=False,
    remove_unused_columns=False)

# --- Metric Functions (unchanged, F1 implementation corrected for multi-class) ---
def sql_tokenize(s):
    s = s.lower().strip()
    for tok in ["(", ")", ",", ";"]:
        s = s.replace(tok, f" {tok} ")
    return s.split()

def compute_exact_match(pred, gold):
    return 1.0 if pred.strip().lower() == gold.strip().lower() else 0.0

# NOTE: For token-level F1 in text-to-SQL, 'micro' average is standard
def compute_f1_micro(pred, gold):
    pred_tokens = sql_tokenize(pred)
    gold_tokens = sql_tokenize(gold)
    all_tokens = list(set(pred_tokens + gold_tokens))
    pred_vec = [1 if t in pred_tokens else 0 for t in all_tokens]
    gold_vec = [1 if t in gold_tokens else 0 for t in all_tokens]
    if sum(gold_vec) == 0:
        return 1.0
    # Use average="micro" for correct token-level F1 calculation
    return f1_score(gold_vec, pred_vec, average="micro")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    if isinstance(predictions, (list, tuple)):
        if isinstance(predictions[0], (list, tuple)):
            predictions = [p[0] for p in predictions]
    else:
        predictions = [predictions]

    if isinstance(labels, (list, tuple)):
        labels = [l if isinstance(l, np.ndarray) else np.array(l) for l in labels]
    else:
        labels = [labels]

    all_exact_scores = []
    all_f1_scores = []

    for pred_chunk, label_chunk in zip(predictions, labels):
        pred_ids = np.argmax(pred_chunk, axis=-1)
        decoded_preds_chunk = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

        label_chunk = np.where(label_chunk != -100, label_chunk, tokenizer.pad_token_id)
        decoded_labels_chunk = tokenizer.batch_decode(label_chunk, skip_special_tokens=True)

        chunk_exact = [compute_exact_match(p, l) for p, l in zip(decoded_preds_chunk, decoded_labels_chunk)]
        # Use the corrected F1 function
        chunk_f1 = [compute_f1_micro(p, l) for p, l in zip(decoded_preds_chunk, decoded_labels_chunk)]

        all_exact_scores.extend(chunk_exact)
        all_f1_scores.extend(chunk_f1)

    return {
        "exact_match": np.mean(all_exact_scores),
        "token_f1": np.mean(all_f1_scores)
    }

# --- Trainer Initialization (using the small validation probe set) ---

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val_probe, # <--- Uses the small 2k probe set
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)
trainer.model_accepts_loss_kwargs = False
trainer.train()

# Save the LoRA adapter
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

# ------------------------------------------------------------------
#  INFERENCE AND KPI CREATION (Using the full 20k validation set)
# ------------------------------------------------------------------

# Ensure drive is mounted for paths to work (redundant, but good practice in a notebook)
try:
    drive.mount('/content/drive', force_remount=True)
except:
    pass

# --- Model Loading ---

print(f"\nUsing device: {device}")

# Load tokenizer and base model (Note: Base model should be the same as trained with)
tokenizer = AutoTokenizer.from_pretrained(SAVE_DIR)
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if device == "cuda" else torch.float32,
    low_cpu_mem_usage=True
)
base_model.to(device)
base_model.eval()

# Load PeFT adapter on top of the base model
model = PeftModel.from_pretrained(base_model, SAVE_DIR)
model.to(device)
model.eval()

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    max_length=128,
    return_tensors="pt"
)

print("Model loaded successfully!")

# --- Data Loading (Full 20k Validation Set) ---

# Use the full, large validation set for the final metric calculation
df_new_sample_full = pd.read_csv(VAL_DATA_PATH).sample(n=20000, random_state=42)
print(f"Loaded {len(df_new_sample_full)} full validation samples for final evaluation.")

gold_sql = df_new_sample_full["sql"].tolist()
gold_complexity = df_new_sample_full["sql_complexity"].tolist()

dataset_new = Dataset.from_pandas(df_new_sample_full)

# Preprocess function for inference
def preprocess_inference(batch):
    inputs = batch["input_text"]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    return model_inputs

tokenized_new = dataset_new.map(
    preprocess_inference,
    batched=True,
    remove_columns=["input_text"]
)

# --- Metric Functions (re-used/defined for context) ---

def is_executable_sql(sql_str, dialect='mysql'):
    try:
        parsed = parse_one(sql_str, dialect=dialect)
        return 1.0
    except Exception:
        return 0.0

# --- Generation config ---
gen_config = GenerationConfig.from_model_config(model.config)
gen_config.num_beams = 1
gen_config.max_new_tokens = 128
gen_config.do_sample = False
gen_config.pad_token_id = tokenizer.pad_token_id

# --- Run Inference (Batch) ---
predictions = []
batch_size = 64

print("\nStarting inference on full validation set...")
start_time = time.time()

for i in range(0, len(tokenized_new), batch_size):
    end_i = min(i + batch_size, len(tokenized_new))

    batch_dict = {}
    for col in tokenized_new.column_names:
        if col not in ["sql", "sql_complexity"]:
            batch_dict[col] = tokenized_new[col][i:end_i]

    features = [
        {col: batch_dict[col][j] for col in batch_dict}
        for j in range(len(batch_dict[list(batch_dict.keys())[0]]))
    ]
    batch = data_collator(features)

    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            generation_config=gen_config
        )

    gen_sql = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    predictions.extend([
        sql.split(tokenizer.eos_token)[0] if tokenizer.eos_token else sql
        for sql in gen_sql
    ])

    if (i // batch_size + 1) % 10 == 0:
        elapsed = time.time() - start_time
        print(f"Processed {end_i}/{len(tokenized_new)} samples in {elapsed:.1f}s "
              f"({(end_i/len(tokenized_new))*100:.1f}%)")

end_time = time.time()
print(f"\nInference complete in {end_time - start_time:.1f}s")

# --- Final Evaluation ---

exact_scores = [compute_exact_match(p, g) for p, g in zip(predictions, gold_sql)]
f1_scores = [compute_f1_micro(p, g) for p, g in zip(predictions, gold_sql)]
valid_syntax_scores = [is_executable_sql(p) for p in predictions]

token_f1 = np.mean(f1_scores)
exact_match = np.mean(exact_scores)
syntax_validity = np.mean(valid_syntax_scores)

print(f"\n=== FINAL Evaluation on Full Validation Set ({len(predictions)} samples) ===")
print(f"Token F1: {token_f1:.4f}")
print(f"Exact Match: {exact_match:.4f}")
print(f"Syntax Validity: {syntax_validity:.4f}")

# --- Bias Analysis (Re-using logic) ---

print("\n=== Bias Analysis: Performance by sql_complexity ===")

df_eval_final = df_new_sample_full.reset_index(drop=True).copy()
df_eval_final["pred_sql"] = predictions
df_eval_final["exact_match"] = exact_scores
df_eval_final["token_f1"] = f1_scores
df_eval_final["valid_syntax"] = valid_syntax_scores

# Per-bucket performance: EM and token F1
per_bucket = (
    df_eval_final
    .groupby("sql_complexity")[["exact_match", "token_f1", "valid_syntax"]]
    .agg(["mean", "count"])
    .sort_values(("token_f1", "mean"), ascending=False)
)

print("\nPer-sql_complexity performance (mean over Full Validation Set):")
print(per_bucket)

Using device: cuda
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

...

Mounted at /content/drive

Using device: cuda
Model loaded successfully!
Loaded 20000 full validation samples for final evaluation.


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]


Starting inference on full validation set...


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Processed 640/20000 samples in 77.4s (3.2%)
Processed 1280/20000 samples in 154.2s (6.4%)
Processed 1920/20000 samples in 231.3s (9.6%)
Processed 2560/20000 samples in 308.4s (12.8%)
Processed 3200/20000 samples in 385.5s (16.0%)
Processed 3840/20000 samples in 462.2s (19.2%)
Processed 4480/20000 samples in 539.2s (22.4%)
Processed 5120/20000 samples in 615.7s (25.6%)
Processed 5760/20000 samples in 692.6s (28.8%)
Processed 6400/20000 samples in 769.6s (32.0%)
Processed 7040/20000 samples in 846.6s (35.2%)
Processed 7680/20000 samples in 923.6s (38.4%)
Processed 8320/20000 samples in 1000.5s (41.6%)
Processed 8960/20000 samples in 1077.6s (44.8%)
Processed 9600/20000 samples in 1155.1s (48.0%)
Processed 10240/20000 samples in 1232.1s (51.2%)
Processed 10880/20000 samples in 1309.2s (54.4%)
Processed 11520/20000 samples in 1386.2s (57.6%)
Processed 12160/20000 samples in 1463.3s (60.8%)
Processed 12800/20000 samples in 1540.2s (64.0%)
Processed 13440/20000 samples in 1616.9s (67.2%)
Pro